In [31]:
import pandas as pd


Задание 1

In [32]:
df=pd.read_csv('incidents.csv')
df.describe()


,team,id,events_count,crit_rate
count,28000.000000,28000.000000,28000.000000,28000.000000
mean,938.714286,551109.067000,50.327571,5.051400
std,27.209803,259009.437919,28.888730,2.856262
min,911.000000,100018.000000,1.000000,0.100000
25%,920.000000,326425.250000,25.000000,2.620000
50%,931.500000,551577.500000,50.000000,5.030000
75%,943.250000,774950.250000,75.000000,7.540000
max,995.000000,999966.000000,100.000000,10.000000


In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28000 entries, 0 to 27999
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   team                28000 non-null  int64  
 1   id                  28000 non-null  int64  
 2   name                28000 non-null  object 
 3   description         28000 non-null  object 
 4   type                28000 non-null  object 
 5   events_count        28000 non-null  int64  
 6   crit_rate           28000 non-null  float64
 7   assets_id           28000 non-null  object 
 8   vulnerabilities_id  28000 non-null  object 
 9   start_time          28000 non-null  object 
 10  end_time            28000 non-null  object 
dtypes: float64(1), int64(3), object(7)
memory usage: 2.3+ MB


In [34]:
df.describe(include=["object"])

,name,description,type,assets_id,vulnerabilities_id,start_time,end_time
count,28000,28000,28000,28000,28000,28000,28000
unique,24,19871,6,5698,12976,27513,27993
top,Удаление учетных данных,"Обнаружена активность ""Удаление учетных данных...",Повышение прав,['50126'],['1021'],2023-10-02 00:11:51.546,2023-10-10 00:19:42.171
freq,2419,33,4734,37,307,2,2


In [35]:
df.sort_values(by="crit_rate", ascending=False).head()

,team,id,name,description,type,events_count,crit_rate,assets_id,vulnerabilities_id,start_time,end_time
23600,941,149112,SSH,"Обнаружена активность ""SSH"" с уязвимостями 102...",Боковое передвижение,49,10.0,"['32105', '48670']","['1023', '1024', '1020', '1011']",2023-09-25 00:11:53.865,2023-09-25 00:18:53.865
24877,942,298625,Управление учетными данными,"Обнаружена активность ""Управление учетными дан...",Повышение прав,88,10.0,['35820'],"['1001', '1003', '1017']",2023-10-03 00:11:54.725,2023-10-03 00:24:54.725
27109,945,579752,Доступ к файлам,"Обнаружена активность ""Доступ к файлам"" с уязв...",Действия на целевом устройстве,93,10.0,['51650'],"['1023', '1024', '1007']",2023-09-26 00:11:55.861,2023-09-26 00:15:55.861
663,911,201664,Токсичные бинарники,"Обнаружена активность ""Токсичные бинарники"" с ...",Закрепление,3,10.0,['43360'],"['1024', '1023']",2023-10-05 00:11:41.458,2023-10-05 00:46:41.458
16444,925,776621,Маскировка,"Обнаружена активность ""Маскировка"" с уязвимост...",Избежание обнаружения,95,10.0,['15358'],"['1024', '1015']",2023-09-18 00:11:50.136,2023-09-18 01:08:50.136


Задание 2

In [36]:
df['SYTKI'] = pd.cut(pd.to_datetime(df.start_time).dt.hour, # бининг по началу времени(часа в фрейме)
                         bins = [0, 6, 12, 18, 24],    # значения интервалов
                         labels = ['night', 'morning', 'afternoon', 'evening'],  #названия интервалов
                         right = False,  #исключения самого правого значения
                     )
df.groupby(['name', 'SYTKI']).agg({'SYTKI': 'count'})

C:\Users\79037\AppData\Local\Temp\ipykernel_10088\128457294.py:6: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby(['name', 'SYTKI']).agg({'SYTKI': 'count'})


SYTKI
name                        SYTKI           
Pass-the-Hash               night        973
                            morning        0
                            afternoon      0
                            evening        0
Pass-the-Ticket             night        961
...                                      ...
Эксплуатация уязвимостей    evening        0
Эксплуатация уязвимостей ОС night        955
                            morning        0
                            afternoon      0
                            evening        0

[96 rows x 1 columns]

In [37]:
# Задание 3
import requests
API_URL = 'https://d5d9e0b83lurt901t9ue.apigw.yandexcloud.net'
def get_assets_os(assets_id):
      assets_os = set()
      for asset_id in assets_id.split(','):
       assets_os.add(requests.get(f'{API_URL}/get-asset-by-id', params={'asset-id': asset_id}).json().get('result', {}).get('os'))
      return str(assets_os)

test_df = df.head(100).copy()
test_df.loc[:, 'assets_id'] = test_df['assets_id'].apply(lambda x: str(x).replace("[", '').replace("]", '').replace("'", ''))
test_df.loc[:, 'asset_os'] = test_df['assets_id'].apply(get_assets_os)
result = test_df.groupby('asset_os').agg({'events_count': 'mean', 'crit_rate': 'mean'})
result

,events_count,crit_rate
asset_os,,
"{'Cisco IOS', 'Red Hat Enterprise 8'}",32.000000,5.420000
"{'Cisco IOS', 'Windows Server 2022'}",64.000000,8.810000
{'Cisco IOS'},49.000000,5.313000
"{'Red Hat Enterprise 8', 'Windows Server 2022'}",96.000000,5.250000
{'Red Hat Enterprise 8'},57.352941,4.232353
"{'Ubuntu 20.04', 'Red Hat Enterprise 8'}",84.000000,6.410000
"{'Ubuntu 20.04', 'Windows Server 2022'}",65.666667,5.583333
{'Ubuntu 20.04'},50.173913,5.860870
"{'Windows 10 Pro', 'Cisco IOS'}",19.500000,3.040000


Задание 4

In [38]:
df['start_time'] = pd.to_datetime(df.start_time)
df['end_time'] = pd.to_datetime(df.end_time)
df_copy = df.copy()
df_copy['relation'] = df.apply(lambda x: x['events_count'] / (
   (x['end_time'] - x['start_time']).total_seconds() # считает отношение кол-во среднего кол-во инцидентов к времени инциденты
), axis=1)
df_copy

df_copy.sort_values('relation', inplace=True) # сортирует отношения
median = df.head(5)['crit_rate'].median() # выводит медиану первых пяти

df_copy.loc[df.crit_rate > median] # оставляем только те, что выше медианы

,team,id,name,description,type,events_count,crit_rate,assets_id,vulnerabilities_id,start_time,end_time,SYTKI,relation
12231,921,792767,Доступ к файлам,"Обнаружена активность ""Доступ к файлам"" с уязв...",Действия на целевом устройстве,1,8.74,['58263'],"['1007', '1018', '1011', '1021']",2023-09-27 00:11:47.947,2023-09-27 01:11:47.947,night,0.000278
16523,925,906582,Эксплуатация уязвимостей,"Обнаружена активность ""Эксплуатация уязвимосте...",Проникновение,1,9.97,['89929'],['1004'],2023-09-25 00:11:50.187,2023-09-25 01:11:50.187,night,0.000278
24345,942,134748,Сбор данных,"Обнаружена активность ""Сбор данных"" с уязвимос...",Действия на целевом устройстве,1,6.13,['89589'],"['1010', '1014', '1012', '1001']",2023-09-13 00:11:54.399,2023-09-13 01:11:54.399,night,0.000278
9246,993,800786,Фишинг,"Обнаружена активность ""Фишинг"" с уязвимостями ...",Проникновение,1,7.59,['90851'],"['1011', '1017', '1004']",2023-09-13 00:11:46.049,2023-09-13 01:11:46.049,night,0.000278
15781,924,408052,Сбор данных,"Обнаружена активность ""Сбор данных"" с уязвимос...",Действия на целевом устройстве,1,4.94,['31350'],"['1013', '1007']",2023-09-17 00:11:49.773,2023-09-17 01:10:49.773,night,0.000282
...,...,...,...,...,...,...,...,...,...,...,...,...,...
11365,995,965461,Сбор данных,"Обнаружена активность ""Сбор данных"" с уязвимос...",Действия на целевом устройстве,98,7.92,['67023'],['1001'],2023-10-03 00:11:47.503,2023-10-03 00:12:47.503,night,1.633333
19575,932,419803,Web-инъекции,"Обнаружена активность ""Web-инъекции"" с уязвимо...",Проникновение,99,3.52,['92003'],"['1019', '1015']",2023-10-05 00:11:51.727,2023-10-05 00:12:51.727,night,1.650000
21554,934,436919,Сбор данных,"Обнаружена активность ""Сбор данных"" с уязвимос...",Действия на целевом устройстве,99,9.64,['39459'],['1019'],2023-10-01 00:11:52.722,2023-10-01 00:12:52.722,night,1.650000
25742,943,620320,Маскировка,"Обнаружена активность ""Маскировка"" с уязвимост...",Избежание обнаружения,100,5.83,['20638'],"['1021', '1016', '1023']",2023-09-20 00:11:55.172,2023-09-20 00:12:55.172,night,1.666667


Задание 5

In [39]:
user_access_levels = {
    "admin": 1,
    "user123": 0.4,
    "dbadmin": 0.8,
    "guest": 0.2,
    "developer": 0.7,
    "tester": 0.75,
    "analyst": 0.6,
    "operator": 0.5,
    "manager": 0.65,
    "consultant": 0.55,
}

def get_level_access_by_asset_id(assets_id):
    users_levels = []
    for asset_id in assets_id.split(','):
        user = requests.get(f'{API_URL}/get-asset-by-id', params={'asset-id': asset_id}).json().get('result', {}).get('account_name') # получаем названием аккаунта
        users_levels.append(user_access_levels[user]) # заполняем список значениями
    return sum(users_levels) / len(users_levels) # возвращаем средние значение

# to reduce requests count
test_df = df.head(100).copy()
test_df.loc[:, 'assets_id'] = test_df['assets_id'].apply(lambda x: str(x).replace("[", '').replace("]", '').replace("'", ''))
test_df.loc[:, 'avg_access_level'] = test_df['assets_id'].apply(get_level_access_by_asset_id) # создаем столбец со средними значениями
# test_df.loc[:, 'asset_os']
result_df = test_df.groupby('name').apply(lambda group: group['','',''].corr()) # таблица корреляции группировка по типам инцидента

result_df.drop(result_df[(result_df['avg_access_level'] == -1.0)].index, inplace=True)
result_df.drop(result_df[(result_df['avg_access_level'] == 1.0)].index, inplace=True)
result_df[(result_df['avg_access_level'].abs() == result_df['avg_access_level'].abs().max())].index # корреляция друг с другом

result_df[(result_df['avg_access_level'].abs() == result_df['avg_access_level'].abs().max())]

KeyError: ('', '', '')